Unzip Data from 42

In [1]:
import zipfile
import pickle
import numpy as np
import pandas as pd
import os

directory = '../data/data.zip'
output_directory = '../data/data/'

# os.listdir(output_directory)
if not os.path.isdir(output_directory):
    os.makedirs(output_directory)
import zipfile
with zipfile.ZipFile(directory, 'r') as zip_ref:
    zip_ref.extractall(output_directory)

Interpolate Input data to make fixed length and get the required frequency domain

In [3]:
import os
import pickle
import pandas as pd
import numpy as np
from collections import Counter
path = '../data/data/'

from scipy import interpolate,signal
def interpolate_data(df):
    df = df[df[:,0].argsort(),:]
    x = df[:,0]
    y = df[:,1:]
    f = interpolate.interp1d(x, y,axis=0,fill_value='extrapolate',kind='linear')
    x_new = np.linspace(df[0,0],df[0,0]+20,500)
    y_new = f(x_new)
    return y_new    

def filter_data_bandpass(X,
                Fs=25,
                low_cutoff=.4,
                high_cutoff=3.0,
                filter_order=65):
    """
    Bandpass Filter of single channel

    :param X: input data
    :param Fs: sampling freq.
    :param low_cutoff: low passband
    :param high_cutoff: high passband
    :param filter_order: no of taps in FIR filter

    :return: filtered version of input data
    """
    X1 = X.reshape(-1,1)
    X1 = signal.detrend(X1,axis=0,type='constant')
    b = signal.firls(filter_order,np.array([0,low_cutoff-.1, low_cutoff, high_cutoff ,high_cutoff+.5,Fs/2]),np.array([0, 0 ,1 ,1 ,0, 0]),
                     np.array([100*0.02,0.02,0.02]),fs=Fs)
    X2 = signal.convolve(X1.reshape(-1),b,mode='same')
    return X2

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    data = data.reshape(-1,1)
    data = signal.detrend(data,axis=0,type='constant').reshape(-1)
    b, a = butter_highpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

def highpass_filter(df,cutoff=4,fs=25,order=3):
    df[:,0] = butter_highpass_filter(df[:,0], cutoff, fs, order)
    df[:,1] = butter_highpass_filter(df[:,1], cutoff, fs, order)
    df[:,2] = butter_highpass_filter(df[:,2], cutoff, fs, order)
    return df

X = []
y = []
for f in list(os.listdir(path)):
    if f[0]=='.':
        continue
    data = pickle.load(open(path+f,'rb'))
    data['data'] = data['data'].apply(lambda a:interpolate_data(a))
    data['data'] = data['data'].apply(lambda a:highpass_filter(a).reshape(1,-1,9))
    X.append(np.concatenate(data['data'].values))
    y.extend(list(data['user']))

X = np.concatenate(X)

y = np.array(y).reshape(-1,1)

from sklearn.preprocessing import OneHotEncoder

y_final = OneHotEncoder().fit_transform(y).todense()

X.shape,y_final.shape

pickle.dump([X,y_final],open('../data/xyhighpass.p','wb'),protocol=4)